In [4]:
import io, os, sys, types
import graphlab as gl
import mysql.connector
from mysql.connector import errorcode
import import_ipynb
import Content_Based_Recommender as C_B_R
import pyodbc
import Conn_E_SQL as Conn
import Creacion_Vectores as C_V
import Normalizacion as Norm
import Recommenders as Recom

#Atributos de BD
host="recomdb.c4v5c6aergun.us-west-2.rds.amazonaws.com"
user="imaginex"
passwd="6^b{X2Du)Qf6"
database="RecomDB"
table= "CBF"

#Atributos de XML o consola

Type=2

#Atributos de Vivienda

Vector_BD_Vivienda= Conn.Conexion_BD(host,user,passwd,database,table)
apartment_file_processed= gl.SFrame(C_V.Creacion_Vector_Apartamento(Vector_BD_Vivienda))
apartment_file_normalized=gl.SFrame(Norm.norm_apartment_matrix(apartment_file_processed))
Vector_IDF=[]
Vector_IDF=Norm.Inv_DF(apartment_file_processed)

#Atributos de Cliente

clientfile="C:\\Users\\user\\Documents\\Sistema de Recomendacion\\Single_Client_raw.csv"
client_file_processed=gl.SFrame(C_V.Creacion_Vector_Cliente(clientfile))
client_array=client_file_processed["ID_C"]
client_number=client_array[0]
zona_array=client_file_processed["Zona"]
zona_number=zona_array[0]

#Atributos de Clientes

Vector_Clientes_BD= Conn.Conexion_BD_clients (host, user, passwd, database, table)
clients_SF_processed=gl.SFrame(C_V.Creacion_BD_Clientes(Vector_Clientes_BD))

#Atributos de Rating

Vector_Ratings_BD=Conn.Conexion_BD_rating(client_number, host, user, passwd, database, table)
client_rating=C_V.Creacion_Matrix_Rating(Vector_Ratings_BD)

#Atributos de Ratings

Vector_Ratings_BD=Conn.Conexion_BD_ratings(host, user, passwd, database, table)
clients_ratings=C_V.Creacion_Matrix_Rating(Vector_Ratings_BD)


#Funcion para entrenar sistema de recomendacion segun el tipo que escoja el cliente

def Recommender_Type (Type):
    
    Values=[]
    
    if Type==1:
        
        a=C_B_R.content_based_recommender()
        b=C_B_R.content_based_recommender()
        c=C_B_R.content_based_recommender()
        d=C_B_R.content_based_recommender()
        b=a.recommender_calculation(apartment_file_normalized,client_file_processed,Vector_IDF,client_number)
        d=c.recommender_matrix(b, client_file_processed, client_number, apartment_file_processed,apartment_file_normalized)
        for x,y,z in zip(d["ID_C"],d["ID_V"],d["Recommender_Score"]):
            cbr_Values=(x,y,z)
            Values.append(cbr_Values)
    
    if Type==2:
        e=Recom.recommenders()
        f=e.item_ranking(apartment_file_processed,client_number,zona_number,client_rating)
        for x,y,z in zip(f["ID_C"],f["ID_V"],f["score"]):
            fact_Values=(x,y,z)
            Values.append(fact_Values)    
      
    if Type==3:
        i=Recom.recommenders()
        j=i.non_personalized_ranking(apartment_file_processed,clients_SF_processed,clients_ratings,client_number, zona_number)
        for x,y in zip(j["ID_V"],j["score"]):
            nonpers_values=(x,y)
            Values.append(nonpers_values)
    
    return Values


def Conexion_S_BD (host, user, passwd, database, table, Type ):
    
    Vector_BD=[]
    
    #Conexión a la base MySQL con Manejo de Excepciones

    try:
        mydb = mysql.connector.connect(  host=host,  user=user,  passwd=passwd,  database=database, 
                                   auth_plugin='mysql_native_password')
            
        mycursor = mydb.cursor()
        
        if(Type==1):
            query_ID=("INSERT INTO cbf_r (ID_C,ID_V, Recommender_Score) VALUES(%s,%s,%s)")
            mycursor.executemany(query_ID,Recommender_Type(Type))                   
            mydb.commit()
        if(Type==2):
            query_ID=("INSERT INTO fact_r (ID_C, ID_V, score) VALUES(%s,%s,%s)")
            mycursor.executemany(query_ID,Recommender_Type(Type))
            mydb.commit()
        if(Type==3):
            query_ID=("INSERT INTO nonpers_r (ID_V, score) VALUES(%s,%s)")
            mycursor.executemany(query_ID,Recommender_Type(Type))
            mydb.commit()
        

    except mysql.connector.Error as err:

        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Usuario o Contrasena equivocada")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("La BD no existe")
        else:
            print(err)
    else:
        mydb.close()
    
Conexion_S_BD(host, user, passwd, database, table, Type)
    
    
    
    

Quieres ver apartamentos fuera de tu zona de escogencia?
1


Finished parsing file C:\Users\user\Documents\Sistema de Recomendacion\Single_Client_raw.csv

Parsing completed. Parsed 1 lines in 0.02696 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long,long,long,long,long,long,long,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\user\Documents\Sistema de Recomendacion\Single_Client_raw.csv

...

Recsys training: model = popularity

Preparing data set.

Data has 4 observations with 10 users and 41 items.

Data prepared in: 0.046874s

4 observations to process; with 41 unique items.

# 